In [1]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

%useLatestDescriptors
%use lets-plot

class Main

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data

In [2]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val vulnCollection = db.getCollection<BsonDocument>("mergedVuln")
val vulnClientCollection = db.getCollection<BsonDocument>("mergedVulnClients")

In [3]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [4]:
import org.nield.kotlinstatistics.median as kmdn

val vulnCveGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val cve = it["cve_ref"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        (cve to gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavToClasses.count())
println(vulnCveGavToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavToClasses.map { it.value.size }.average())
println(vulnCveGavToClasses.map { it.value.size }.kmdn())

7618
1
51
2.0490942504594383
1.0


In [5]:
val vulnCveGavPairToClasses = vulnClientCollection.find().useCursor { blk ->
    blk.map {
        val dep_gav = it["dep_gav"]!!.asString().value
        val client_gav = it["client_gav"]!!.asString().value
        val cve = it["cve"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = vulnCveGavToClasses[cve to dep_gav]!!
        
        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        Triple(cve, dep_gav, client_gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavPairToClasses.count())
println(vulnCveGavPairToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavPairToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavPairToClasses.map { it.value.size }.average())
println(vulnCveGavPairToClasses.map { it.value.size }.kmdn())
println(vulnCveGavPairToClasses.keys.map { it.second }.toSet().count()) // are no client for some of the libs in our dataset

85889
1
42
2.0847605630523116
1.0
2290


In [6]:
println(vulnCveGavPairToClasses.map { it.key.third }.toSet().size)

40677


In [7]:
import scripts.partitionDepGraph.dagP.loadDagpPartitionInfo

val partitionInfo = vulnCveGavToClasses.keys.map {
    it.second to loadDagpPartitionInfo(it.second)
}.toMap()

partitionInfo.count()

3809

In [8]:
import common.DefaultGraph
import io.github.classgraph.ClassInfoList
import java.io.File
import java.net.URLClassLoader
import org.jgrapht.Graph
import org.jgrapht.graph.DefaultEdge
import org.jgrapht.graph.builder.GraphTypeBuilder

fun loadDepGraph(gav: String): DefaultGraph = 
    scripts.exportDepGraphs.loadDepGraphFromCache(gav)

fun loadClassListInfo(gav: String): ClassInfoList =
    scripts.exportDepGraphs.loadClassInfoListFromCache(gav)

fun loadVertexInfo(gav: String) =
    scripts.exportDepGraphs.loadVertexInfo(gav)

fun loadDepGraph(depGav: String, clientGav: String): DefaultGraph = 
    scripts.exportPairDepGraphs.loadDepGraphFromCache(depGav, clientGav)

fun loadClassListInfo(depGav: String, clientGav: String): ClassInfoList =
    scripts.exportPairDepGraphs.loadClassInfoListFromCache(depGav, clientGav)

fun loadVertexInfo(depGav: String, clientGav: String) =
    scripts.exportPairDepGraphs.loadVertexInfo(depGav, clientGav)

In [9]:
import com.google.common.collect.Queues

/**
 * returns the depths of each visited node from the starting node
 */
fun bfsOnDepGraph(graph: DefaultGraph, startNode: String): Map<String, Int> {
    val expanded = mutableSetOf<String>()
    val depthMap = mutableMapOf<String, Int>()
    val queue = Queues.newArrayDeque<String>()
    queue.add(startNode)
    depthMap[startNode] = 0
    while (!queue.isEmpty()) {
        val node = queue.pop()
        expanded.add(node)
        val parentDepth = depthMap[node]!! 
        graph
            .outgoingEdgesOf(node)
            .map { e -> graph.getEdgeTarget(e) }
            .filterNot { expanded.contains(it) }
            .forEach { v ->
                if (depthMap.containsKey(v)) {
                    depthMap[v] = min(depthMap[v]!!, parentDepth + 1)
                } else {
                    depthMap[v] = parentDepth + 1;
                }
                queue.add(v)
            }
    }
    return depthMap
}

fun addFakeSourceToNodes(graph: DefaultGraph, startingNodes: Set<String>, fakeNode: String) {
    graph.addVertex(fakeNode)
    startingNodes.forEach { v ->
        graph.addEdge(fakeNode, v)
    }
}

fun createCondensedGraphFromMapping(
    originalGraph: DefaultGraph,
    atomToCondensedNode: Map<String, String>,
): DefaultGraph {
    val condensedGraph =
        GraphTypeBuilder
            .directed<String, DefaultEdge>()
            .allowingMultipleEdges(false)
            .allowingSelfLoops(false)
            .edgeClass(DefaultEdge::class.java)
            .weighted(true)
            .buildGraph()
    
    atomToCondensedNode.values.toSet().forEach {
        condensedGraph.addVertex(it)
    }
    
    originalGraph.edgeSet().forEach { e ->
        val source = atomToCondensedNode[originalGraph.getEdgeSource(e)]
        if (source == null) {
            println("source was null: ${originalGraph.getEdgeSource(e)} -> ${originalGraph.getEdgeTarget(e)}")
            throw NullPointerException()
        }
        val target = atomToCondensedNode[originalGraph.getEdgeTarget(e)]
        if (target == null) {
            println("target was null: ${originalGraph.getEdgeSource(e)} -> ${originalGraph.getEdgeTarget(e)}")
            throw NullPointerException()
        }
        
        if (source == target) return@forEach
        
        if (!condensedGraph.containsEdge(source, target)) {
            condensedGraph.addEdge(source, target)
        } else {
            val edge = condensedGraph.getEdge(source, target)
            val weight = condensedGraph.getEdgeWeight(edge)
            condensedGraph.setEdgeWeight(edge, weight + 1)
        }
    }
    
    return condensedGraph
}

In [10]:
import io.exoquery.fansi.toStr
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.runBlocking
import org.jetbrains.kotlinx.dataframe.math.median
import org.jgrapht.graph.EdgeReversedGraph
import java.lang.reflect.Modifier

data class VulnAnalysisInfo(
    val depClassCount: Int,
    val depVulnClassCount: Int,
    val clientClassCount: Int,
    val depPublicClassCount: Int,
    val clientPublicClassCount: Int,
    val clientVulnClassCount: Int,
    val clientVulnPublicClassCount: Int,
    val depVulnPublicClassCount: Int,
    val referenedPublicClassCount: Int,
    val vulnModulesUsedByClientCount: Int,
    val safeModulesUsedByClientCount: Int,
    val depModuleCount: Int,
    val vulnDepPublicClassesUsedByClientCount: Int,
    val metadataVulnModulesUsedByClientCount: Int,
    val metadataSafeModulesUsedByClientCount: Int,
    val clientAndDepPublicVulnClassCountBeforeModularization: Int,
    val clientAndDepPublicVulnClassCountAfterModularization: Int,
    val depPublicVulnClassCountBeforeModularization: Int,
    val depPublicVulnClassCountAfterModularization: Int,
    val depPartCount: Int,
    val depPartsUsedByClientCount: Int,
    val medianSizeOfModulesUsedByClient: Double,
    val meanSizeOfModulesUsedByClient: Double,
    val isClientUsingDep: Boolean,
    ) {
    val depVulnRatio = depVulnClassCount.toDouble() / depClassCount.toDouble()
    val depApiSurfaceRatio = depPublicClassCount.toDouble() / depClassCount.toDouble()
    val depVulnApiSurfaceRatio = depVulnPublicClassCount.toDouble() / depPublicClassCount.toDouble()
    
    val clientVulnRatio = clientVulnClassCount.toDouble() / clientClassCount.toDouble()
    val clientApiSurfaceRatio = clientPublicClassCount.toDouble() / clientClassCount.toDouble()
    val clientVulnApiSurfaceRatio = clientVulnPublicClassCount.toDouble() / clientPublicClassCount.toDouble()
}

System.gc()

@kotlinx.coroutines.ExperimentalCoroutinesApi fun _vulnInfo() = Dispatchers.IO.limitedParallelism(32).let { dispatcher ->
    vulnCveGavPairToClasses.asSequence().windowed(512, 512, true).flatMapIndexed { batch, w ->
        runBlocking {
            System.gc()
            println("processing batch $batch")
            w.map {
                async(dispatcher) {
                    val depGav = it.key.second
                    val clientGav = it.key.third
                    val vulnClasses = it.value
                    
                    val depGraph = loadDepGraph(depGav, clientGav)
                    
                    val vertexInfo = loadVertexInfo(depGav, clientGav)
                    val depVertexInfo = loadVertexInfo(depGav)
                    val clientVertexInfo = vertexInfo.filterNot { depVertexInfo.containsKey(it.key) }.toMap()
                    
                    // sanity check if we get similar results with random vuln roots
                    // val vulnClasses = depVertexInfo.keys.shuffled().take(it.value.size).toSet()
                    
                    val depPublicClasses =
                        depGraph.vertexSet().filter { v -> depVertexInfo.containsKey(v) && Modifier.isPublic(vertexInfo[v]!!) }
                    
                    val publicApiUsages = depGraph.edgeSet().map { e ->
                        val s = depGraph.getEdgeSource(e)
                        val t = depGraph.getEdgeTarget(e)
                        if (clientVertexInfo.keys.contains(s) && depVertexInfo.keys.contains(t)) s to t
                        else null
                    }.filterNotNull()
                    
                    // if (publicApiUsages.size == 0) {
                    //     // so many cases that the dep is in the pom of the client but actually is not using it at all! 
                    //     return@async Result.failure(Exception("no public api usage!"))
                    // }
                    
                    val depPublicClassesUsedByClient = publicApiUsages.map { it.second }.toSet()
                    
                    // class-level propagation
                    val revDepGraph = EdgeReversedGraph(depGraph)  // shallow!
                    addFakeSourceToNodes(revDepGraph, vulnClasses, "<fake source>")
                    val distToNearestVuln = mutableMapOf<String, Int>()
                    bfsOnDepGraph(revDepGraph, "<fake source>").let { m ->
                        m.map { it.key to it.value - 1 }.forEach { (v, d) ->
                            if (distToNearestVuln.containsKey(v)) {
                                distToNearestVuln[v] = min(distToNearestVuln[v]!!, d)
                            } else {
                                distToNearestVuln[v] = d
                            }
                        }
                    }
                    distToNearestVuln.remove("<fake source>")
                    revDepGraph.removeVertex("<fake source>")
                    
                    val allVulnClasses = distToNearestVuln.keys.toSet()
                    val allClientVulnClasses = allVulnClasses.intersect(clientVertexInfo.keys)
                    val clientPublicClasses = clientVertexInfo.keys.filter { v ->
                        Modifier.isPublic(clientVertexInfo[v]!!)
                    }
                    val publicClientVulnClasses = allClientVulnClasses.intersect(clientPublicClasses)
                    
                    val depVulnClasses = distToNearestVuln.keys.filter { v ->
                        depVertexInfo.containsKey(v)
                    }.toSet()
                    
                    val depVulnPublicClasses = distToNearestVuln.keys.filter { v ->
                        depVertexInfo.containsKey(v) && Modifier.isPublic(depVertexInfo[v]!!)
                    }.toSet()
                    
                    val vulnDepPublicClassesUsedByClient = depPublicClassesUsedByClient.intersect(depVulnPublicClasses)
                    
                    // val clientVulnRoots = publicApiUsages.filter { depVulnClasses.contains(it.second) }.map { it.first }
                    
                    val depPartitionInfo = loadDagpPartitionInfo(depGav)
                    val depVulnParts = depPartitionInfo.partToVertices.filter { (p, vl) ->
                        vl.intersect(depVulnClasses).isNotEmpty()
                    }
                    
                    val depSafeParts = depPartitionInfo.partToVertices.filterNot { (p, vl) ->
                        depVulnParts.keys.contains(p)
                    }
                    
                    val depVulnPartsUsedByClient =
                        // depVulnParts.filter { (p, vl) -> vl.intersect(vulnDepPublicClassesUsedByClient).isNotEmpty() }
                        depVulnParts.filter { (p, vl) -> vl.intersect(depPublicClassesUsedByClient).isNotEmpty() }
                    
                    val depSafePartsUsedByClient =
                        depSafeParts.filter { (p, vl) -> vl.intersect(depPublicClassesUsedByClient).isNotEmpty() }
                    
                    // meta-data/module level propagation
                    val metadataVulnRootParts = depPartitionInfo.partToVertices.filter { (p, vl) ->
                        vl.intersect(vulnClasses).isNotEmpty()
                    }
                    
                    // println("here?")
                    
                    // client classes + dep modules
                    val moduleDepGraph = createCondensedGraphFromMapping(
                        depGraph,
                        depPartitionInfo.vertexToPart.map { (v, p) -> v to p.toString() }.toMap() + clientVertexInfo
                            .map { it.key to it.key }
                            .toMap() // client nodes should not be condensed 
                    )
                    
                    val distToNearestMetadataVuln = mutableMapOf<String, Int>()
                    
                    val revModuleDepGraph = EdgeReversedGraph(moduleDepGraph)
                    addFakeSourceToNodes(revModuleDepGraph, metadataVulnRootParts.map { it.key.toString() }.toSet(), "<fake source>")
                    bfsOnDepGraph(revModuleDepGraph, "<fake source>").let { m ->
                        m.map { it.key to it.value - 1 }.forEach { (v, d) ->
                            if (distToNearestMetadataVuln.containsKey(v)) {
                                distToNearestMetadataVuln[v] = min(distToNearestMetadataVuln[v]!!, d)
                            } else {
                                distToNearestMetadataVuln[v] = d
                            }
                        }
                    }
                    distToNearestMetadataVuln.remove("<fake source>")
                    revModuleDepGraph.removeVertex("<fake source>")
                    
                    val depPartsUsedByClient = depPartitionInfo.partToVertices.filter { (p, vl) ->
                        vl.intersect(depPublicClassesUsedByClient).isNotEmpty()
                    }.keys
                    
                    val metadataVulnParts: Set<Int> =
                        depPartitionInfo.partToVertices.keys.filter { distToNearestMetadataVuln.containsKey(it.toString()) }.toSet()
                    
                    val depMetadataVulnPartsUsedByClient = metadataVulnParts.filter { p ->
                        depPartitionInfo.partToVertices[p]!!.intersect(depPublicClassesUsedByClient).isNotEmpty()
                    }.toSet()
                    
                    val depMetadataSafePartsUsedByClient = depPartsUsedByClient.filterNot { p ->
                        metadataVulnParts.contains(p)
                    }.toSet()
                    
                    // sanity check
                    if (depPartsUsedByClient.size != depMetadataVulnPartsUsedByClient.size + depMetadataSafePartsUsedByClient.size) {
                        error("depPartsUsedByClient.size != depMetadataVulnPartsUsedByClient.size + depMetadataSafePartsUsedByClient.size")
                    }
                    
                    if (depPartsUsedByClient.size != depVulnPartsUsedByClient.size + depSafePartsUsedByClient.size) {
                        error("depPartsUsedByClient.size != depVulnPartsUsedByClient.size + depSafePartsUsedByClient.size")
                    }
                    
                    // api surface before and after removing unused modules
                    
                    val publicUsedDepVulnClassesBeforeModularization = allVulnClasses.filter {
                        depPartitionInfo.vertexToPart.containsKey(it)
                    }.filter { Modifier.isPublic(depVertexInfo[it]!!) }.toSet()
                    
                    val publicUsedDepVulnClassesAfterModularization = allVulnClasses.filter {
                        val part = depPartitionInfo.vertexToPart[it]
                        return@filter if (part == null) false else depMetadataVulnPartsUsedByClient.contains(part)
                    }.filter { Modifier.isPublic(depVertexInfo[it]!!) }.toSet()
                    
                    // vuln public classes in the client and referenced modules (pre-modularization)
                    val publicVulnClassesBeforeModularization =
                        publicClientVulnClasses + publicUsedDepVulnClassesBeforeModularization
                    
                    // vuln public classes in the client and referenced modules (post-modularization)
                    val publicVulnClassesAfterModularization = 
                        publicClientVulnClasses + publicUsedDepVulnClassesAfterModularization
                    
                    val res = VulnAnalysisInfo(
                        depClassCount = depVertexInfo.keys.size,
                        depPublicClassCount = depPublicClasses.size,
                        depVulnPublicClassCount = depVulnPublicClasses.size,
                        depVulnClassCount = depVulnClasses.size,
                        clientClassCount = clientVertexInfo.keys.size,
                        clientPublicClassCount = clientPublicClasses.size,
                        clientVulnClassCount = allClientVulnClasses.size,
                        clientVulnPublicClassCount = publicClientVulnClasses.size,
                        referenedPublicClassCount = depPublicClassesUsedByClient.size,
                        vulnModulesUsedByClientCount = depVulnPartsUsedByClient.size,
                        safeModulesUsedByClientCount = depSafePartsUsedByClient.size,
                        depModuleCount = depPartitionInfo.partitionCount,
                        vulnDepPublicClassesUsedByClientCount = vulnDepPublicClassesUsedByClient.size,
                        
                        metadataVulnModulesUsedByClientCount = depMetadataVulnPartsUsedByClient.size,
                        metadataSafeModulesUsedByClientCount = depMetadataSafePartsUsedByClient.size,
                        
                        clientAndDepPublicVulnClassCountBeforeModularization = publicVulnClassesBeforeModularization.size,
                        clientAndDepPublicVulnClassCountAfterModularization = publicVulnClassesAfterModularization.size,
                        
                        depPublicVulnClassCountBeforeModularization = publicUsedDepVulnClassesBeforeModularization.size,
                        depPublicVulnClassCountAfterModularization = publicUsedDepVulnClassesAfterModularization.size,
                        
                        isClientUsingDep = publicApiUsages.size == 0,
                        
                        depPartCount = depPartitionInfo.partitionCount,
                        depPartsUsedByClientCount = depPartsUsedByClient.size,
                        
                        medianSizeOfModulesUsedByClient = if (depPartsUsedByClient.size == 0) 0.0 else depPartsUsedByClient.map { p -> depPartitionInfo.partToVertices[p]!!.size.toDouble() }.median(),
                        meanSizeOfModulesUsedByClient = if (depPartsUsedByClient.size == 0) 0.0 else depPartsUsedByClient.map { p -> depPartitionInfo.partToVertices[p]!!.size }.average(),
                    )
                    
                    return@async Result.success(res)
                }
            }.toList().awaitAll().toList()
        }.toList().filter { it.isSuccess }.map { it.getOrNull()!! }.toList()
    }.toList()
}

val vulnInfo = _vulnInfo()
vulnInfo.count()

processing batch 0
processing batch 1
processing batch 2
processing batch 3
processing batch 4
processing batch 5
processing batch 6
processing batch 7
processing batch 8
processing batch 9
processing batch 10
processing batch 11
processing batch 12
processing batch 13
processing batch 14
processing batch 15
processing batch 16
processing batch 17
processing batch 18
processing batch 19
processing batch 20
processing batch 21
processing batch 22
processing batch 23
processing batch 24
processing batch 25
processing batch 26
processing batch 27
processing batch 28
processing batch 29
processing batch 30
processing batch 31
processing batch 32
processing batch 33
processing batch 34
processing batch 35
processing batch 36
processing batch 37
processing batch 38
processing batch 39
processing batch 40
processing batch 41
processing batch 42
processing batch 43
processing batch 44
processing batch 45
processing batch 46
processing batch 47
processing batch 48
processing batch 49
processing

85889

In [11]:
fun Double.format(digits: Int) =
    "%.${digits}f".format(this)

fun Float.format(digits: Int) =
    "%.${digits}f".format(this)

In [12]:
println("${vulnCveGavPairToClasses.count() - vulnInfo.count()}")
println("${(vulnCveGavPairToClasses.count() - vulnInfo.count()).toDouble() / vulnCveGavPairToClasses.count()} ")

0
0.0 


In [13]:
println("${vulnInfo.map { it.depPartsUsedByClientCount / it.depPartCount.toDouble() }.kmdn()}")

0.25


In [14]:
println("${vulnInfo.filter { it.isClientUsingDep }.size}")
println("${vulnInfo.filter { it.isClientUsingDep && it.clientVulnClassCount == 0 }.size.toDouble() / vulnInfo.filter { it.clientVulnClassCount == 0 }.size.toDouble()}")

30804
0.5422380256649475


In [15]:
println("${vulnInfo.filter { it.vulnModulesUsedByClientCount == 0 }.size}")
println("${vulnInfo.filter { it.vulnModulesUsedByClientCount != 0 && it.clientVulnClassCount == 0 }.size.toDouble()}")
println("${vulnInfo.filter { it.vulnModulesUsedByClientCount == 0 }.size.toDouble()}")
println("${vulnInfo.filter { it.clientVulnClassCount == 0 }.size.toDouble()}")
// println("${vulnInfo.filter { it.vulnModulesUsedByClientCount == 0 }.size.toDouble() / vulnInfo.filter { it.clientVulnClassCount == 0 }.size.toDouble()}")

println("${vulnInfo.filter { it.clientVulnClassCount == 0 && it.metadataVulnModulesUsedByClientCount == 0 }.size.toDouble() / vulnInfo.filter { it.clientVulnClassCount == 0 }.size.toDouble()}")

47330
9479.0
47330.0
56809.0
0.7733105669876252


In [16]:
println("${vulnInfo.filter { it.vulnModulesUsedByClientCount == 0 && it.metadataVulnModulesUsedByClientCount > 0 }.size.toDouble() / vulnInfo.filter { it.clientVulnClassCount == 0 }.size.toDouble()}")

0.059832068862328155


In [17]:
println("${vulnInfo.map { (it.clientAndDepPublicVulnClassCountAfterModularization - it.clientAndDepPublicVulnClassCountBeforeModularization).toDouble() / it.clientAndDepPublicVulnClassCountBeforeModularization }.filterNot { it.isNaN() }.average() }")

println("${vulnInfo.map { (it.depPublicVulnClassCountAfterModularization - it.depPublicVulnClassCountBeforeModularization).toDouble() / it.depPublicVulnClassCountBeforeModularization }.filterNot { it.isNaN() }.average() }")

-0.6416636759561605
-0.6471478616719499


In [18]:
println("how many clients are meta-data safe? (they won't get the security alerts at meta-data level")
println("${vulnInfo.filter { it.metadataVulnModulesUsedByClientCount == 0 }.size}")
println("${vulnInfo.filter { it.metadataVulnModulesUsedByClientCount == 0 }.size.toDouble() / vulnInfo.count()}")

println("how many clients are not using any meta-data safe modules? (they don't get any benefit)")
println("${vulnInfo.filter { it.metadataSafeModulesUsedByClientCount == 0 }.size}")
println("${vulnInfo.filter { it.metadataSafeModulesUsedByClientCount == 0 }.size.toDouble() / vulnInfo.count()}")

println("how many client use all the modules?")
println("${vulnInfo.filter { it.metadataVulnModulesUsedByClientCount + it.metadataSafeModulesUsedByClientCount == it.depModuleCount }.size}")
println("${vulnInfo.filter { it.metadataVulnModulesUsedByClientCount + it.metadataSafeModulesUsedByClientCount == it.depModuleCount }.size.toDouble() / vulnInfo.count()}")

// println("${vulnInfo.filter { it.vulnModulesUsedByClientCount != 0 && it.clientVulnClassCount == 0 }.size.toDouble()}")
// println("${vulnInfo.filter { it.vulnModulesUsedByClientCount == 0 }.size.toDouble()}")
// println("${vulnInfo.filter { it.clientVulnClassCount == 0 }.size.toDouble()}")
// println("${vulnInfo.filter { it.vulnModulesUsedByClientCount == 0 }.size.toDouble() / vulnInfo.filter { it.clientVulnClassCount == 0 }.size.toDouble()}")

how many clients are meta-data safe? (they won't get the security alerts at meta-data level
43931
0.5114857548696573
how many clients are not using any meta-data safe modules? (they don't get any benefit)
54211
0.631175121377592
how many client use all the modules?
3673
0.04276449836416771


In [19]:
println("Out of meta-data vulnerable clients, how many are 'actually vuln'?")
println("${vulnInfo.filter { it.vulnModulesUsedByClientCount > 0 }.size}")
println("${vulnInfo.filter { it.vulnModulesUsedByClientCount > 0 }.size / vulnInfo.size.toDouble()}")
// ans:
println("${vulnInfo.filter { it.metadataVulnModulesUsedByClientCount > 0 && it.clientVulnClassCount > 0 }.size.toDouble() / vulnInfo.filter { it.metadataVulnModulesUsedByClientCount > 0 }.count()}")


Out of meta-data vulnerable clients, how many are 'actually vuln'?
38559
0.44893991081512186
0.6930740264073597


In [20]:
vulnInfo.filter { it.vulnModulesUsedByClientCount == 0 && it.clientVulnClassCount != 0 }.size

0

In [21]:
vulnInfo.filter { it.clientVulnClassCount == 0 }.size

56809

In [22]:
val safeClientCount = vulnInfo.filter { it.clientVulnClassCount == 0 }.count()
val vulnClientCount = vulnInfo.filter { it.clientVulnClassCount > 0 }.count()
println("safe clients: ${safeClientCount} ratio: ${(safeClientCount.toDouble() / vulnInfo.count()).format(2)}")
println("vuln clients: ${vulnClientCount} ratio: ${(vulnClientCount.toDouble() / vulnInfo.count()).format(2)}")

safe clients: 56809 ratio: 0.66
vuln clients: 29080 ratio: 0.34


In [23]:
val nonSafeClientsInfo = vulnInfo.filter { it.clientPublicClassCount > 0 && it.clientVulnClassCount > 0 }
nonSafeClientsInfo.count()

29067

 if (vuln) clients remove vuln modules, how much of their references to external libs are going be removed?

In [24]:
import org.jetbrains.kotlinx.dataframe.math.median

println("${vulnInfo.filter { it.clientVulnClassCount > 0 }.map { it.vulnDepPublicClassesUsedByClientCount }.median()}")
println("${vulnInfo.filter { it.clientVulnClassCount > 0 }.map { it.vulnDepPublicClassesUsedByClientCount }.average()}")
println("${vulnInfo.filter { it.clientVulnClassCount > 0 }.map { it.vulnDepPublicClassesUsedByClientCount / it.referenedPublicClassCount.toDouble() }.median()}")
println("${vulnInfo.filter { it.clientVulnClassCount > 0 }.map { it.vulnDepPublicClassesUsedByClientCount / it.referenedPublicClassCount.toDouble() }.average()}")

1
5.278198074277854
0.6
0.5848539241553482


In [25]:
import org.nield.kotlinstatistics.median as kmdn
import org.jetbrains.kotlinx.dataframe.math.median as mdn
println("${vulnInfo.map { it.depVulnClassCount.toDouble() / it.depClassCount.toDouble() }.kmdn() }")

println("${vulnInfo.map { it.depPartCount }.mdn()}")
println("${vulnInfo.map { it.depPartCount }.average()}")

0.037037037037037035
4
6.222426620405407


In [26]:
val p =
    letsPlot(
        mapOf("clientVulnClassCount" to vulnInfo.filter { it.clientVulnClassCount > 0 }.map { it.clientVulnClassCount.coerceAtMost(500) })
    ) { x = "clientVulnClassCount" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) +
            xlim(limits = Pair(0, 501))
p

-20 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 220 
 
 
 
 
 
 
 
 
 240 
 
 
 
 
 
 
 
 
 260 
 
 
 
 
 
 
 
 
 280 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 320 
 
 
 
 
 
 
 
 
 340 
 
 
 
 
 
 
 
 
 360 
 
 
 
 
 
 
 
 
 380 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 420 
 
 
 
 
 
 
 
 
 440 
 
 
 
 
 
 
 
 
 460 
 
 
 
 
 
 
 
 
 480 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 520 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 5,500 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 6,500 
 
 
 
 
 
 
 7,000 
 
 
 
 
 
 
 
 count 
 
 
 
 
 clientVulnClassCount

In [27]:
val p =
    letsPlot(
        mapOf("avgModuleSizes" to vulnInfo.filter { it.meanSizeOfModulesUsedByClient == 0.0 }.map { it.meanSizeOfModulesUsedByClient.toDouble() / it.depClassCount * 100 })
    ) { x = "avgModuleSizes" } + ggsize(1200, 400) +
    geomHistogram(alpha = .3, binWidth = 1, center = .5) +
    xlim(limits = Pair(0, 100)) + themeGrey() + xlab("")

ggsave(p, "/tmp/graph.png")

p

0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 90 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 
 
 25,000 
 
 
 
 
 
 
 
 
 30,000 
 
 
 
 
 
 
 
 
 
 count